In [5]:
import pandas as pd
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
captureSize = 35.8 #for tmb claculation
import seaborn as sns

In [6]:
! mkdir -p /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2
itrp_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2'

In [7]:
dfm1 = pd.read_excel('./Freeman_raw/Supplementary_Table_4.xlsx',sheet_name = '8. Pre_batch_correction', index_col=0)
dfm2 = pd.read_excel('./Freeman_raw/Supplementary_Table_6.xlsx',sheet_name = '2. Pre_batch_correction', index_col=0)

dfm = dfm1.join(dfm2)
df_mRNA_tpm = 2**dfm-1

In [8]:
df_mRNA_tpm.shape

(19820, 457)

In [9]:
allen = pd.read_csv('./Allen_clinical.csv',index_col=0)
allen.cohort = 'Allen'
gide = pd.read_csv('./Gide_clinical.csv',index_col=0)
hugo = pd.read_csv('./Hugo_clinical.csv',index_col=0)
liu = pd.read_csv('./Liu_clinical.csv',index_col=0)
mgh = pd.read_csv('./MGH_clinical.csv',index_col=0)
riaz = pd.read_csv('./Riaz_clinical.csv',index_col=0)

In [10]:
dfc = pd.concat([ liu, gide,allen, mgh, hugo, riaz])
dfc['RECIST_Response'] = dfc.RECIST.map({'CR':'R', 'PR':'R', 'SD':'NR', 'PD':'NR', 'R':'R', 'NR':'NR'})
dfc.groupby(['cohort', 'RECIST_Response']).size().unstack()

RECIST_Response,NR,R
cohort,,
Allen,31,7
Gide,33,40
Hugo,12,14
Liu,66,39
MGH,25,15
Riaz,39,10


In [11]:
dfc.groupby(['cohort', 'Freeman_response']).size().unstack()

Freeman_response,NR,R
cohort,,
Allen,26,13
Gide,27,46
Hugo,12,14
Liu,66,41
MGH,25,15
Riaz,41,10


In [12]:
def correction(x, cr_os_min):
    if x.RECIST == 'PR':
        if x.Overall_survival > cr_os_min:
            res = 'R'
        else:
            res = 'NR'
    else:
        res = x.Freeman_response
    return res

def cohort_label_correction(dfc):
    cr_os_min = dfc[dfc.RECIST == 'CR'].Overall_survival.min()
    cr_os_min = cr_os_min
    label_list=[]
    for i in range(len(dfc)):
        x = dfc.iloc[i]
        res = correction(x, cr_os_min)
        label_list.append(res)
    lb = pd.Series(label_list, index = dfc.index)
    lb.name = 'Freeman_response_correction'
    return lb

In [13]:
dfc['Freeman_response_correction'] = dfc['Freeman_response']

dfc_gide = dfc[dfc.cohort == 'Gide']
dfc_gide.loc[:,'Freeman_response_correction'] = cohort_label_correction(dfc_gide)

dfc_hugo = dfc[dfc.cohort == 'Hugo']
dfc_hugo.loc[:,'Freeman_response_correction'] = cohort_label_correction(dfc_hugo)

dfc.loc[dfc_hugo.index] = dfc_hugo
dfc.loc[dfc_gide.index] = dfc_gide

In [14]:
dfcf = dfc[dfc.is_in_FreeMan]

In [15]:
dfc.groupby(['cohort', 'Freeman_response_correction']).size().unstack()

Freeman_response_correction,NR,R
cohort,,
Allen,26,13
Gide,32,41
Hugo,17,9
Liu,66,41
MGH,25,15
Riaz,41,10


In [16]:
dfcf.groupby(['cohort', 'Freeman_response_correction']).size().unstack()

Freeman_response_correction,NR,R
cohort,,
Allen,26,13
Gide,32,41
Hugo,17,9
Liu,66,41
MGH,24,14
Riaz,41,10


In [17]:
dfm1_correct = pd.read_excel('./Freeman_raw/Supplementary_Table_4.xlsx',sheet_name = '7. Post_batch_correction', index_col=0)
dfm2_correct = pd.read_excel('./Freeman_raw/Supplementary_Table_6.xlsx',sheet_name = '1. Post_batch_correction', index_col=0)

dfm_correct = dfm1_correct.join(dfm2_correct)
dfm_correct.columns = dfm_correct.columns.map(lambda x:x.replace('.', '-'))
# df_mRNA_tpm_correct = 2**dfm-1

In [18]:
df_mRNA_tpm_pre = df_mRNA_tpm[dfcf.index]
df_mRNA_tpm_pre_correct = dfm_correct[dfcf.index]

In [19]:
sidx = df_mRNA_tpm_pre_correct.index.map(lambda x:x.split('.')[0])
df_mRNA_tpm_pre = df_mRNA_tpm_pre.loc[sidx]
df_mRNA_tpm_pre.index = df_mRNA_tpm_pre_correct.index

In [302]:
# dfc.to_pickle('./00_final_data/ITRP.clinical.pkl')
# df_mRNA_tpm_pre.to_pickle('./00_final_data/ITRP.mRNA.pkl')
# df_mRNA_tpm_pre_correct.to_pickle('./00_final_data/ITRP.mRNA.corrected.pkl')

In [23]:
dfc.to_pickle(os.path.join(itrp_data_path, 'PATIENTS.TABLE'))
df_mRNA_tpm_pre.to_pickle(os.path.join(itrp_data_path, 'TPM.TABLE'))
df_mRNA_tpm_pre_correct.to_pickle(os.path.join(itrp_data_path, 'TPM_CORRECTION.TABLE'))